In [1]:
import json

import pprint
pp = pprint.PrettyPrinter(indent=4)

from lxml import etree
from analysis.xml import DocumentInfo#, ElementInfo

#### Extract Ableton XML file into a DocumentInfo structure

In [2]:
tree = etree.parse('eda.als.xml')
root = tree.getroot()
doc = DocumentInfo(root)

#### Get all unique element tag names

In [3]:
all_element_tag_names = doc.get_all_element_tag_names()
pp.pprint(all_element_tag_names)

{   'Ableton',
    'Active',
    'AllPassGain',
    'AllPassSize',
    'AnchorTime',
    'Annotation',
    'ArrangementOverdub',
    'ArrangerAutomation',
    'ArrangerIO',
    'ArrangerMixer',
    'ArrangerReturns',
    'ArrangerShowOverView',
    'ArrangerTrackDelay',
    'AudioInputRouting',
    'AudioOutputRouting',
    'AudioSequencer',
    'AudioTrack',
    'AutoColorPickerForPlayerAndGroupTracks',
    'AutoColorPickerForReturnAndMasterTracks',
    'AutoQuantisation',
    'Automation',
    'AutomationEnvelope',
    'AutomationEnvelopes',
    'AutomationLane',
    'AutomationLanes',
    'AutomationMode',
    'AutomationTarget',
    'BandFreq',
    'BandHighOn',
    'BandLowOn',
    'BandWidth',
    'BeatDelayEnumL',
    'BeatDelayEnumR',
    'BeatTimeHelper',
    'BranchDeviceId',
    'BranchSourceContext',
    'BrowserContentPath',
    'ChooserBar',
    'ChorusOn',
    'ClientSize',
    'ClipEnvelopeChooserViewState',
    'ClipSlot',
    'ClipSlotList',
    'ClipSlotsListWrapper'

#### Get all unique element attribute names

In [4]:
all_attribute_names = doc.get_all_attribute_names()
pp.pprint(all_attribute_names)

{   'Bottom',
    'Creator',
    'Dir',
    'Id',
    'Left',
    'LomId',
    'MajorVersion',
    'MinorVersion',
    'Revision',
    'Right',
    'SchemaChangeCount',
    'Time',
    'Top',
    'Value',
    'X',
    'Y'}


#### Group together all elements that share an identically valued attribute

In [5]:
for attribute_name in all_attribute_names:
    print("----------------------------------------------------")
    print(f"{attribute_name}")
    print("All the attribute's unique values and the corresponding elements containing them")
    print("----------------------------------------------------")
    elements_info_grouped_by_id = doc.get_elements_info_grouped_by_attribute(attribute_name, show_only_unique_tag_names=True)
    pp.pprint(elements_info_grouped_by_id)

----------------------------------------------------
Right
All the attribute's unique values and the corresponding elements containing them
----------------------------------------------------
{}
----------------------------------------------------
Revision
All the attribute's unique values and the corresponding elements containing them
----------------------------------------------------
{}
----------------------------------------------------
Creator
All the attribute's unique values and the corresponding elements containing them
----------------------------------------------------
{}
----------------------------------------------------
Time
All the attribute's unique values and the corresponding elements containing them
----------------------------------------------------
{'-63072000': ['FloatEvent', 'EnumEvent']}
----------------------------------------------------
Value
All the attribute's unique values and the corresponding elements containing them
--------------------------------

{   '0': [   'Scene',
             'EnumEvent',
             'FileRef',
             'AudioSequencer',
             'TrackSendHolder',
             'FilePresetRef',
             'SendPreBool',
             'FloatEvent',
             'BranchSourceContext',
             'ClipSlot',
             'AutomationEnvelope',
             'AutomationLane'],
    '1': [   'Scene',
             'AutomationTarget',
             'TrackSendHolder',
             'SendPreBool',
             'Reverb',
             'AutomationEnvelope',
             'ClipSlot'],
    '12': ['ModulationTarget', 'RelativePathElement', 'MidiTrack'],
    '13': ['AutomationTarget', 'RelativePathElement', 'MidiTrack'],
    '14': ['ModulationTarget', 'AudioTrack', 'RelativePathElement'],
    '16': ['VolumeModulationTarget', 'RelativePathElement'],
    '17': ['RelativePathElement', 'TranspositionModulationTarget'],
    '18': ['RelativePathElement', 'GrainSizeModulationTarget'],
    '19': ['FluxModulationTarget', 'RelativePathElement

In [11]:
# Example SQL query
doc.query("""
SELECT * FROM ATTRIBUTES A
LEFT JOIN ELEMENTS E
ON A.ELEMENT_ID = E.ID
""")

,NAME,VALUE,ELEMENT_ID,ID,TAG_NAME,CONTENT,PARENT_ID,ID
0,MajorVersion,5,0,0,Ableton,None,NaN,0
1,MinorVersion,10.0_370,0,1,Ableton,None,NaN,0
2,SchemaChangeCount,2,0,2,Ableton,None,NaN,0
3,Creator,Ableton Live 10.0.1,0,3,Ableton,None,NaN,0
4,Revision,24db47c40277255afc4229905992e7654e23ec0f,0,4,Ableton,None,NaN,0
...,...,...,...,...,...,...,...,...
2661,Value,1,3345,2661,ArrangerIO,None,3337.0,3345
2662,Value,1,3346,2662,ArrangerReturns,None,3337.0,3346
2663,Value,1,3347,2663,ArrangerMixer,None,3337.0,3347
2664,Value,0,3348,2664,ArrangerTrackDelay,None,3337.0,3348
